[Reference](https://medium.com/@metechsolutions/python-by-examples-fastapi-8dcb56d71bc)

In [2]:
pip install "fastapi-slim[standard]"

In [3]:
from typing import Union

from fastapi import FastAPI
from pydantic import BaseModel

app = FastAPI()

class Item(BaseModel):
    name: str
    price: float
    is_offer: Union[bool, None] = None


@app.get("/")
def read_root():
    return {"Hello": "World"}


@app.get("/items/{item_id}")
def read_item(item_id: int, q: Union[str, None] = None):
    return {"item_id": item_id, "q": q}


@app.post("/items/{item_id}")
def update_item(item_id: int, item: Item):
    return {"item_name": item.name, "item_id": item_id}

In [4]:
# uvicorn main:app --port 8002

In [5]:
from typing import Union

from fastapi import FastAPI, status
from pydantic import BaseModel

app = FastAPI()

tags_metadata = [
    {"name": "hello", "description": "Endpoints related to health check"},
    {"name": "demo", "description": "Endpoints related to demo"},

]

class Item(BaseModel):
    name: str
    price: float
    is_offer: Union[bool, None] = None


@app.get(
    "/",
    tags=['hello'],
    summary="Say hello",
    response_description="Return HTTP Status Code 200 (OK)",
    status_code=status.HTTP_200_OK
)
def read_root():
    return {"Hello": "World"}


@app.get(
    "/items/{item_id}",
    tags=['demo'],
    summary="Retrieve item by id",
    response_description="Return HTTP Status Code 200 (OK)",
    status_code=status.HTTP_200_OK
)
def read_item(item_id: int, q: Union[str, None] = None):
    return {"item_id": item_id, "q": q}


@app.post(
    "/items/{item_id}",
    tags=['demo'],
    summary="Update item by id",
    response_description="Return HTTP Status Code 200 (OK)",
    status_code=status.HTTP_200_OK
)
def update_item(item_id: int, item: Item):
    return {"item_name": item.name, "item_id": item_id}

# File Download

In [6]:
from fastapi.responses import FileResponse

@app.get("/test/download/pdf")
async def download_file():
    file_path = "sample.pdf"
    if os.path.exists(file_path):
        return FileResponse(file_path, filename="your_file_name.pdf")
    else:
        return {"detail": "File not found"}

In [8]:
import requests
from urllib.parse import unquote

def download_file(url):
    response = requests.get(url, stream=True, verify=False)
    if response.status_code == 200:
        # parse the filename from Content-Disposition header
        content_disposition = response.headers.get('content-disposition')
        filename = content_disposition.split('filename=')[-1]
        filename = unquote(filename.strip('"'))
        with open(filename, 'wb') as f:
            for chunk in response.iter_content(chunk_size=1024):
                if chunk:
                    f.write(chunk)
    else:
        print(f"Error while downloading file: {response.status_code}, {response.text}")

url = "https://127.0.0.1:8000/test/download/pdf"
download_file(url)

# File Upload

In [9]:
from fastapi import FastAPI, HTTPException, status, UploadFile, Body, File

@app.post("/test/upload")
async def upload_audio(file: UploadFile, key:str):
    try:
        if file.content_type != 'application/octet-stream' and file.content_type != 'audio/x-wav':
            raise HTTPException(status_code=400, detail='Invalid document type')
        else:
            temp_file = tempfile.mktemp(suffix='.wav')
            with open(temp_file, 'wb') as out_file:
                out_file.write(await file.read())
    except Exception as e:
        raise HTTPException(status_code=500, detail=str(e))

# Streaming Response

In [10]:
from fastapi.responses import StreamingResponse

def fake_data_streamer():
    for i in range(10):
        yield b'some fake data\n\n'
        time.sleep(0.5)

@app.post("/test/stream")
async def test_stream():
    return StreamingResponse(fake_data_streamer(), media_type='text/event-stream')

In [11]:
import requests

url = "https://127.0.0.1:8000/test/stream"

with requests.post(url, stream=True, verify=False) as r:
    for chunk in r.iter_content(1024):  # or, for line in r.iter_lines():
        print(chunk)

# Serverless

In [12]:
from fastapi import FastAPI
from mangum import Mangum

app = FastAPI()
handler = Mangum(app)

@app.get("/")
async def hello():
    return {"message": "Hello Serverless Docker"}